<a href="https://colab.research.google.com/github/Menon-Vineet/PPG-Codes/blob/main/Winner%20V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ STEP 1: Install & Import
!pip install -q qwen-vl-utils
from transformers import AutoProcessor, AutoModelForVision2Seq
from qwen_vl_utils import process_vision_info
from PIL import Image
import torch, re
import pandas as pd
from google.colab import files

# ✅ STEP 2: Load Model
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForVision2Seq.from_pretrained(model_id, trust_remote_code=True, torch_dtype=torch.float16).to(device)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# ✅ STEP 3: Upload Images
uploaded = files.upload()
records = []

# ✅ STEP 4: Helper Function — Extract Total Aphids and Area
def extract_total_and_area(text):
    # Match format: total:<count>, area:<leaf_area>
    match = re.search(r"total[:=]?\s*(\d+)[^\d]+area[:=]?\s*(\d+)", text.lower())
    if match:
        total = int(match.group(1))
        area = int(match.group(2))
        density = round(total / area, 4) if area else 0
        return total, area, density
    else:
        return 0, 0, 0

# ✅ STEP 5: Inference Loop
for fname in uploaded:
    print(f"\n🔍 Processing {fname}")
    image = Image.open(fname).convert("RGB")

    # Prompt ONLY for total count and leaf area
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": "Count the total number of aphids on this leaf and estimate the total leaf area in mm². Format your answer exactly like this: total:<aphid_count>, area:<leaf_area>"}
            ]
        }
    ]

    # Tokenize and process input
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)

    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ).to(device)

    # Generate output
    outputs = model.generate(**inputs, max_new_tokens=128)
    output_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    print("📊 Raw Output:", output_text)

    # Extract numbers
    total, area, density = extract_total_and_area(output_text)

    # Append result
    records.append({
        "Image": fname,
        "Total Aphids": total,
        "Leaf Area (mm²)": area,
        "Density (aphids/mm²)": density
    })

# ✅ STEP 6: Export CSV
df = pd.DataFrame(records)
df = df.sort_values("Total Aphids", ascending=False).reset_index(drop=True)
df.to_csv("aphid_totals.csv", index=False)
df


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2160: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Saving q (1).jpeg to q (1) (3).jpeg
Saving q (1).jpg to q (1) (3).jpg
Saving q (2).jpeg to q (2) (4).jpeg
Saving q (2).jpg to q (2) (4).jpg
Saving q (3).jpeg to q (3) (4).jpeg
Saving q (3).jpg to q (3) (5).jpg
Saving q (4).jpeg to q (4) (4).jpeg
Saving q (4).jpg to q (4) (4).jpg
Saving q (5).jpeg to q (5) (4).jpeg
Saving q (5).jpg to q (5) (4).jpg
Saving q (6).jpg to q (6) (4).jpg
Saving q (7).jpg to q (7) (4).jpg

🔍 Processing q (1) (3).jpeg
📊 Raw Output: system
You are a helpful assistant.
user
Count the total number of aphids on this leaf and estimate the total leaf area in mm². Format your answer exactly like this: total:<aphid_count>, area:<leaf_area>
assistant
total:0, area:250000

🔍 Processing q (1) (3).jpg
📊 Raw Output: system
You are a helpful assistant.
user
Count the total number of aphids on this leaf and estimate the total leaf area in mm². Format your answer exactly like this: total:<aphid_count>, area:<leaf_area>
assistant
total:0, area:1500

🔍 Processing q (2) (4).jpeg


,Image,Total Aphids,Leaf Area (mm²),Density (aphids/mm²)
0,q (5) (4).jpeg,500,10000,0.0500
1,q (5) (4).jpg,150,20000,0.0075
2,q (7) (4).jpg,150,2000,0.0750
3,q (4) (4).jpeg,100,2500,0.0400
4,q (6) (4).jpg,25,10000,0.0025
5,q (2) (4).jpeg,25,1000,0.0250
6,q (4) (4).jpg,20,1500,0.0133
7,q (3) (4).jpeg,15,2000,0.0075
8,q (3) (5).jpg,15,20000,0.0008
9,q (2) (4).jpg,0,1500,0.0000


In [2]:

# ✅ STEP 5: Export to CSV
df = pd.DataFrame(records)
df = df.sort_values("Total Aphids", ascending=False).reset_index(drop=True)
df.to_csv("aphid_analysis_output.csv", index=False)
print("\n✅ Saved CSV → aphid_analysis_output.csv")

# Optional download for Colab
from google.colab import files
files.download("aphid_analysis_output.csv")



✅ Saved CSV → aphid_analysis_output.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>